In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
import os

classes = []  # 初始化为空列表

CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))   # 获取当前脚本文件的绝对路径，获取指定路径的目录名
#结合使用可以获取当前脚本文件所在的目录

# 这是一个将边界框坐标从绝对像素值转换为相对于图像尺寸的比例值的函数。
# 函数接受两个参数：size（图像尺寸）和box（边界框坐标）。函数返回一个包含转换后的边界框坐标的元组。

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(image_id):
    in_file = open('./label_xml\%s.xml' % (image_id), encoding='UTF-8')  # 打开输入文件
    out_file = open('./label_txt\%s.txt' % (image_id), 'w')  # 创建输出的txt文件
    tree = ET.parse(in_file)  #解析xml文件
    root = tree.getroot()  # 获取根节点
    size = root.find('size')  #查找标签
    w = int(size.find('width').text)  # 获取图像的高度和宽度
    h = int(size.find('height').text)

    for obj in root.iter('object'):  #遍历xml文件中所有的对象
        cls = obj.find('name').text
        if cls not in classes:
            classes.append(cls)  # 如果类别不存在，添加到classes列表中
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

xml_path = os.path.join(CURRENT_DIR, './label_xml/')

# xml list
img_xmls = os.listdir(xml_path)
for img_xml in img_xmls:
    label_name = img_xml.split('.')[0]
    print(label_name)
    convert_annotation(label_name)

print("Classes:")  # 打印最终的classes列表
print(classes)  # 打印最终的classes列表